In [1]:
from propositional import *
from pprint import pprint

In [2]:
s = "hyp"
ann = Ann(s)
print(type(ann))
print(f"input_str = {s}")
print(ann.build_str())
print(ann.rule=='hyp')
s2 = "or elim 3,4-8,9-14"
ann2 = Ann(s2)
print(ann2.build_str())
print(ann2.rule)
print(ann2.premise)

<class 'propositional.Ann'>
input_str = hyp
rule: hyp
premise: []
True
rule: or elim
premise: [3, (4, 8), (9, 14)]
or elim
[3, (4, 8), (9, 14)]


In [3]:
line = "A imp B .hyp"
label = NodeLabel('formula', line)
print(label.build_str())
print()
print(label)

type: formula
line: A imp B .hyp
formula: A imp B
ann: hyp
is_hyp: True

A imp B	 .hyp


In [4]:
line1 = "A imp B .hyp"
label1 = NodeLabel('formula', line1)
node1 = ProofNode(label1)
line2 = "A .hyp"
label2 = NodeLabel('formula', line2)
node2 = ProofNode(label2)
line3 = "B .imp elim 1,2"
label3 = NodeLabel('formula', line3)
node3 = ProofNode(label3)
line4 = "A .repeat 2"
label4 = NodeLabel('formula', line4)
node4 = ProofNode(label4)
label0 = NodeLabel()
node0 = ProofNode(label0, [node1, node2, node3, node4])
print(node0)

│1. A imp B	 .hyp
│2. A	 .hyp
├─
│3. B	 .imp elim 1,2
│4. A	 .repeat 2



In [5]:
line1 = "A imp B .hyp"
label1 = NodeLabel('formula', line1)
node1 = ProofNode(label1)
line2 = "B imp C .hyp"
label2 = NodeLabel('formula', line2)
node2 = ProofNode(label2)
line3 = "A .hyp"
label3 = NodeLabel('formula', line3)
node3 = ProofNode(label3)
line4 = "B .imp elim 1,3"
label4 = NodeLabel('formula', line4)
node4 = ProofNode(label4)
line5 = "C .imp elim 2,4"
label5 = NodeLabel('formula', line5)
node5 = ProofNode(label5)
line6 = "A imp C .imp intro 3-5"
label6 = NodeLabel('formula', line6)
node6 = ProofNode(label6)
label_root = NodeLabel()
label_sub = NodeLabel()
node_sub = ProofNode(label_sub, [node3, node4, node5])
node_root = ProofNode(label_root, [node1, node2, node_sub, node6])
print(node_root)

│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5



In [6]:
# You may or may not prefix each line with line number.
# The parser can assign the correct line number 
# to each node anyway.

prf_str1 = '''
1. A imp B .hyp
2. B imp C .hyp
proves
  3. A .hyp
  proves
  4. B     .imp elim 1,3
  5. C     .imp elim 2,4
6. A imp C .imp intro 3-5
'''
# or,
prf_str2 = '''
│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5
'''
# or, above two without numbering
prf_str3 = '''
A imp B .hyp
B imp C .hyp
proves
  A .hyp
  proves
  B     .imp elim 1,3
  C     .imp elim 2,4
A imp C .imp intro 3-5
'''
# or,
prf_str4 = '''
│A imp B	 .hyp
│B imp C	 .hyp
├─
││A	 .hyp
│├─
││B	 .imp elim 1,3
││C	 .imp elim 2,4
│A imp C	 .imp intro 3-5
'''
# Convert either string to the following.
'''
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5
'''

for i in range(1, 5):
  prf_str_name = f"prf_str{i}"
  prf_str = eval(prf_str_name)
  print(f"[{prf_str_name}]:")
  lines = get_str_li(prf_str, tabsize=2)
  print_lines(lines)    
  print()

[prf_str1]:
A imp B .hyp
B imp C .hyp
{{
  A .hyp
  B     .imp elim 1,3
  C     .imp elim 2,4
}}
A imp C .imp intro 3-5

[prf_str2]:
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5

[prf_str3]:
A imp B .hyp
B imp C .hyp
{{
  A .hyp
  B     .imp elim 1,3
  C     .imp elim 2,4
}}
A imp C .imp intro 3-5

[prf_str4]:
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5



In [7]:
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp
    proves
    4. B .imp elim 1,3
    5. B or C .or intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B .or C imp intro 2-8'''
lines = get_str_li(prf_str, tabsize=2)
print_lines(lines)


A imp B .hyp
{{
  A or C .hyp
  {{
    A .hyp
    B .imp elim 1,3
    B or C .or intro 4
    C .hyp
    B or C .or intro 6
  }}
  B or C .or elim 2,3-5,6-7
}}
A or C imp B .or C imp intro 2-8


In [9]:
prf_str = '''
1. A .hyp
2. B  .hyp
proves
3. A and B .and intro
'''
print(parse_fitch(prf_str))

│1. A	 .hyp
│2. B	 .hyp
├─
│3. A and B	 .and intro



In [10]:
prf_str = '''
1. A imp B .hyp
2. B imp C .hyp
proves
  3. A .hyp
  proves
  4. B     .imp elim 1,3
  5. C     .imp elim 2,4
6. A imp C .imp intro 3-5
'''
print(parse_fitch(prf_str))

│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5



In [12]:
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp
    proves
    4. B .imp elim 1,3
    5. B or C .or intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B or C .imp intro 2-8'''
print(parse_fitch(prf_str))

│1. A imp B	 .hyp
├─
││2. A or C	 .hyp
│├─
│││3. A	 .hyp
││├─
│││4. B	 .imp elim 1,3
│││5. B or C	 .or intro 4
│││6. C	 .hyp
│││7. B or C	 .or intro 6
││8. B or C	 .or elim 2,3-5,6-7
│5. A or C imp B or C	 .imp intro 2-8



In [13]:
prf_str = '''
1. A .hyp
proves
  2. B .hyp
  proves
    3. C .hyp
    proves
    4. B and c .and intro 2,3
5. B imp (C imp B and C) .imp intro 2-4
'''
print(parse_fitch(prf_str))

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
│││3. C	 .hyp
││├─
│││4. B and c	 .and intro 2,3
│4. B imp C imp B and C	 .imp intro 2-4

